In [1]:
import pandas as pd
import time
import tensorflow as tf


def normalize(dataset):
    seriesIndices = ['hours']
    for i in range(1,31):
        seriesIndices.append(f"hours_l{i}")
    dataset[seriesIndices] = dataset[seriesIndices].apply(lambda x: (x - x.mean()) / (x.std()))
    return dataset

include_fields = ['hours','day_of_week','hours_l1','hours_l2','hours_l3','hours_l4',
                  'hours_l5','hours_l6','hours_l7','hours_l8','hours_l14','hours_l15',
                  'hours_l21','hours_l22','hours_l28','hours_l29','week_perc0','week_perc6']

startTime = time.time()
train = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/training_set.csv",usecols=include_fields).dropna()
val = pd.read_csv("/export/storage_adgandhi/PBJhours_ML/Data/Intermediate/train_test_validation/validation_set.csv",usecols=include_fields).dropna()
print(f"Loaded Train and Validation sets. Time taken: {time.time()-startTime}")

Loaded Train and Validation sets. Time taken: 331.31281065940857


In [2]:
train_inputs, train_labels = train.drop(['hours'], axis=1), train.filter(['hours'])
val_inputs, val_labels = val.drop(['hours'], axis=1), val.filter(['hours'])

#test_inputs, test_labels = test.drop(['hours'], axis=1), test.filter(['hours'])

In [3]:
def expand_one_hot(labels,dataset):
    outList = []
    for label in labels:  
        col = dataset[label]
        ###Generate a dict for all unique values (Don't waste space encoding non important job id's)
        map = {}
        index = 0
        for element in col.unique():
            map[element] = index
            index += 1
        col = col.map(map)
        tensor = tf.one_hot(col,len(col.unique()))
        outList.append(tensor)
        dataset = dataset.drop(columns=[label])
    
    outList.append(dataset)
    output = tf.concat(outList,1)
    return output

train_inputs = expand_one_hot(['day_of_week'],train_inputs)
val_inputs = expand_one_hot(['day_of_week'],val_inputs)
#test_inputs = expand_one_hot(['day_of_week'],test_inputs)

print(train_inputs.shape)

(125197363, 23)


In [4]:
strategy = tf.distribute.MirroredStrategy()
BUFFER_SIZE = 10000
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
BATCH_SIZE_PER_REPLICA = 256
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync
trainSet = tf.data.Dataset.from_tensor_slices((train_inputs,train_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
valSet = tf.data.Dataset.from_tensor_slices((val_inputs,val_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
#testSet = tf.data.Dataset.from_tensor_slices((test_inputs,test_labels)).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(trainSet)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
Number of devices: 1
<BatchDataset shapes: ((None, 23), (None, 1)), types: (tf.float32, tf.float64)>


In [5]:
with strategy.scope():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(1),
    ])
    
    model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

In [6]:
import os
checkpoint_dir = './training_checkpointsEfficientModel'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

def decay(epoch):
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

callbacks = [
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True),
    tf.keras.callbacks.LearningRateScheduler(decay)
]


In [7]:
model.fit(trainSet, epochs=10, callbacks=callbacks, validation_data=valSet)

Epoch 1/10
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
489053/489053 [==============================] - 1464s 3ms/step - loss: 10.1614 - mean_absolute_error: 2.2185 - val_loss: 11.6420 - val_mean_absolute_error: 2.6426
Epoch 2/10
489053/489053 [==============================] - 1340s 3ms/step - loss: 9.9495 - mean_absolute_error: 2.1734 - val_loss: 11.7097 - val_mean_absolute_error: 2.6751
Epoch 3/10
489053/489053 [==============================] - 1158s 2ms/step - loss: 9.9445 - mean_absolute_error: 2.1716 - val_loss: 11.5889 - val_mean_absolute_error: 2.6363
Epoch 4/10
489053/489053 [==============================] - 1152s 2ms/step - loss: 10.0936 - mean_absolute_error: 2.2016 - val_loss: 12.0780 - val_mean_absolute_error: 2.7707
Epoch 5/10
489053/4890

In [ ]:
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
eval_loss, eval_acc = model.evaluate(testSet)

print('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

In [8]:
checkpoint_dir = './training_checkpointsEfficientModel2'
with strategy.scope():
    model2 = tf.keras.Sequential([
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dense(1),
    ])
    
    model2.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [9]:
model2.fit(trainSet, epochs=10, callbacks=callbacks, validation_data=valSet)

Epoch 1/10
489053/489053 [==============================] - 1217s 2ms/step - loss: 10.3062 - mean_absolute_error: 2.2436 - val_loss: 12.2468 - val_mean_absolute_error: 2.7602
Epoch 2/10
489053/489053 [==============================] - 1208s 2ms/step - loss: 10.1203 - mean_absolute_error: 2.2030 - val_loss: 12.4996 - val_mean_absolute_error: 2.7757
Epoch 3/10
489053/489053 [==============================] - 1225s 3ms/step - loss: 10.2035 - mean_absolute_error: 2.2177 - val_loss: 12.3028 - val_mean_absolute_error: 2.7610
Epoch 4/10
489053/489053 [==============================] - 1210s 2ms/step - loss: 10.3202 - mean_absolute_error: 2.2431 - val_loss: 12.6287 - val_mean_absolute_error: 2.8407
Epoch 5/10
489053/489053 [==============================] - 1217s 2ms/step - loss: 10.2502 - mean_absolute_error: 2.2288 - val_loss: 12.5555 - val_mean_absolute_error: 2.8415
Epoch 6/10
489053/489053 [==============================] - 1215s 2ms/step - loss: 10.2063 - mean_absolute_error: 2.2199 - va

In [10]:
checkpoint_dir = './training_checkpointsEfficientModel3'
with strategy.scope():
    model3 = tf.keras.Sequential([
        tf.keras.layers.Dense(64,activation='relu'),
        tf.keras.layers.Dropout(0.4),
        tf.keras.layers.Dense(32,activation='relu'),
        tf.keras.layers.Dense(1),
    ])
    
    model3.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

In [11]:
model3.fit(trainSet, epochs=10, callbacks=callbacks, validation_data=valSet)

Epoch 1/10
489053/489053 [==============================] - 1098s 2ms/step - loss: 10.1540 - mean_absolute_error: 2.2201 - val_loss: 11.7048 - val_mean_absolute_error: 2.7040
Epoch 2/10
489053/489053 [==============================] - 1043s 2ms/step - loss: 9.8547 - mean_absolute_error: 2.1633 - val_loss: 11.7482 - val_mean_absolute_error: 2.6643
Epoch 3/10
489053/489053 [==============================] - 1025s 2ms/step - loss: 9.8349 - mean_absolute_error: 2.1602 - val_loss: 11.8265 - val_mean_absolute_error: 2.6653
Epoch 4/10
489053/489053 [==============================] - 1025s 2ms/step - loss: 10.0304 - mean_absolute_error: 2.1976 - val_loss: 11.7682 - val_mean_absolute_error: 2.7119
Epoch 5/10
489053/489053 [==============================] - 1029s 2ms/step - loss: 9.9575 - mean_absolute_error: 2.1837 - val_loss: 11.9248 - val_mean_absolute_error: 2.7367
Epoch 6/10
489053/489053 [==============================] - 1039s 2ms/step - loss: 9.9435 - mean_absolute_error: 2.1828 - val_lo